# Recherche non informée et informée
Notebook inspiré du chapitre "Solving Problems by Searching" (AIMA).

Dans ce notebook, nous allons illustrer :
- Les recherches non informées (BFS, DFS, etc.).  
- Les recherches informées (Greedy Best First Search, A*, etc.).  

Nous prendrons en exemple un problème de "route-finding" (cheminement sur un graphe) similaire à la carte de la Roumanie proposée dans AIMA. Nous utiliserons:
- Une classe Problem (qui définit l'état initial, l'état but, les actions, etc.).
- Une classe GraphProblem (pour un graphe non orienté ou orienté).
- Plusieurs fonctions d'algorithmes de recherche (BFS, DFS, A*, etc.) adaptées pour visualiser les étapes.

## Technologies et Bibliothèques utilisées

- Python 3
- matplotlib (pour d'éventuelles visualisations)
- networkx (pour la manipulation de graphes)
- collections (deque), heapq, etc.

In [19]:
%pip install matplotlib networkx 

Note: you may need to restart the kernel to use updated packages.


### Imports et Fonctions Utilitaires

Nous allons commencer par importer les bibliothèques nécessaires et définir les fonctions utilitaires.

In [20]:
import math
import sys
import random
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque
%matplotlib inline

# Pour ignorer certains avertissements si nécessaire
import warnings
warnings.filterwarnings("ignore")

# Exemple simplifié de "carte" sous forme de graphe non orienté
class UndirectedGraph:
    def __init__(self, graph_dict=None):
        self.graph_dict = graph_dict or {}
        self.nodes = set()
        for key in self.graph_dict:
            self.nodes.add(key)
            for val in self.graph_dict[key]:
                self.nodes.add(val)
                
    def get(self, a):
        """Renvoie un dictionnaire {voisin: distance} pour le noeud a."""
        return self.graph_dict.get(a, {})
    
    def __repr__(self):
        return f"UndirectedGraph({len(self.nodes)} noeuds)"

# Exemple : carte simplifiée inspirée de la Roumanie dans AIMA :
roumanie_graph = UndirectedGraph({
    'Arad':    {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
    'Zerind':  {'Arad': 75, 'Oradea': 71},
    'Oradea':  {'Zerind': 71, 'Sibiu': 151},
    'Sibiu':   {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu': 80},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Rimnicu': {'Sibiu': 80, 'Pitesti': 97, 'Craiova': 146},
    'Pitesti': {'Rimnicu': 97, 'Bucharest': 101, 'Craiova': 138},
    'Craiova': {'Pitesti': 138, 'Rimnicu': 146, 'Drobeta': 120},
    'Drobeta': {'Craiova': 120, 'Mehadia': 75},
    'Mehadia': {'Drobeta': 75, 'Lugoj': 70},
    'Lugoj':   {'Mehadia': 70, 'Timisoara': 111},
    'Timisoara': {'Lugoj': 111, 'Arad': 118},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Urziceni': 85, 'Giurgiu': 90},
    'Giurgiu':   {'Bucharest': 90},
    'Urziceni':  {'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142},
    'Hirsova':   {'Urziceni': 98, 'Eforie': 86},
    'Eforie':    {'Hirsova': 86},
    'Vaslui':    {'Urziceni': 142, 'Iasi': 92},
    'Iasi':      {'Vaslui': 92, 'Neamt': 87},
    'Neamt':     {'Iasi': 87},
})

## Définition de la classe de problème

Nous définissons la classe de base Problem et la classe GraphProblem pour décrire le problème sur un graphe.

In [21]:
class Problem:
    """
    Classe abstraite d'un problème de recherche:
    - initial : état initial
    - goal : état but
    - actions(state) : renvoie la liste des actions possibles depuis state
    - result(state, action) : renvoie l'état successeur lorsque action est appliquée sur state
    - goal_test(state) : renvoie True si state est un état but
    - path_cost(c, state1, action, state2) : coût pour aller de state1 à state2 via action
    """
    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal
    
    def actions(self, state):
        raise NotImplementedError
    
    def result(self, state, action):
        raise NotImplementedError
    
    def goal_test(self, state):
        return state == self.goal
    
    def path_cost(self, c, state1, action, state2):
        return c + 1
    
    def h(self, node):
        """Heuristique à définir pour les recherches informées."""
        return 0

class GraphProblem(Problem):
    """
    Problème de recherche sur un graphe (non orienté ou orienté).
    """
    def __init__(self, initial, goal, graph):
        super().__init__(initial, goal)
        self.graph = graph

    def actions(self, state):
        """Retourne les voisins du noeud 'state' dans le graphe."""
        return list(self.graph.get(state).keys())
    
    def result(self, state, action):
        """L'action = nom du voisin; le nouvel état est l'action elle-même."""
        return action
    
    def path_cost(self, cost_so_far, A, action, B):
        """Le coût est la distance entre A et B, selon le graphe."""
        return cost_so_far + (self.graph.get(A).get(B) or math.inf)

    def h(self, node):
        """
        Exemple d'heuristique : distance à vol d'oiseau (si on a des coordonnées).
        Ici, on n'a pas de coordonnées pour toutes les villes de la carte,
        Mais si on en avait: on pourrait calculer la distance euclidienne.
        On renvoie 0 par défaut.
        """
        return 0

## Création d'un problème d'itinéraire

Nous allons configurer le problème de recherche : aller de 'Arad' à 'Bucharest' dans notre graphe.

In [22]:
# On définit un problème de recherche : aller de 'Arad' à 'Bucharest'.
romania_problem = GraphProblem('Arad', 'Bucharest', roumanie_graph)

## Recherche non informée

### Recherche en largeur (Breadth-First Search)

Nous implémentons deux variantes :  
- breadth_first_tree_search : ne gère pas les doublons  
- breadth_first_graph_search : gère les états déjà explorés

In [23]:
from collections import deque

def breadth_first_tree_search(problem):
    """
    Recherche en largeur (Tree Search) : ne gère pas la détection des doublons.
    Parcourt le graphe en augmentant progressivement la profondeur.
    """
    frontier = deque([Node(problem.initial)])
    while frontier:
        node = frontier.popleft()  # Premier entré, premier sorti (FIFO)
        if problem.goal_test(node.state):
            return node
        for child in node.expand(problem):
            frontier.append(child)
    return None

def breadth_first_graph_search(problem):
    """
    Recherche en largeur (Graph Search) : gère les états déjà explorés
    pour éviter les répétitions.
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = deque([node])
    explored = set()
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if (child.state not in explored) and (child not in frontier):
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    return None

class Node:
    """
    Noeud dans l'arbre de recherche.
    - state : état
    - parent : noeud parent
    - action : action menant de parent à ce noeud
    - path_cost : coût du chemin depuis la racine
    """
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1
    
    def expand(self, problem):
        """Génère les noeuds fils résultant des actions possibles."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]
    
    def child_node(self, problem, action):
        """Crée un noeud fils à partir d’une action."""
        next_state = problem.result(self.state, action)
        return Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next_state))
    
    def path(self):
        """Retourne la liste de noeuds depuis la racine jusqu’à ce noeud."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def solution(self):
        """Retourne la liste d'actions pour passer de la racine à ce noeud."""
        return [node.action for node in self.path()[1:]]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state
    
    def __hash__(self):
        return hash(self.state)

# Test rapide BFS :
solution_bfs = breadth_first_graph_search(romania_problem)
if solution_bfs:
    print("Chemin (BFS) =", solution_bfs.solution())
    print("Coût =", solution_bfs.path_cost)
else:
    print("Aucune solution trouvée par BFS.")

Chemin (BFS) = ['Sibiu', 'Fagaras', 'Bucharest']
Coût = 450


### Recherche en profondeur (Depth-First Search)

Deux variantes :  
- depth_first_tree_search  
- depth_first_graph_search (évite de revisiter des états déjà explorés)

In [24]:
def depth_first_tree_search(problem):
    """Recherche en profondeur (Version Tree Search)."""
    frontier = [Node(problem.initial)]
    while frontier:
        node = frontier.pop()  # Dernier entré, premier sorti (LIFO)
        if problem.goal_test(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None

def depth_first_graph_search(problem):
    """Recherche en profondeur (Graph Search) : gère les visités."""
    frontier = [Node(problem.initial)]
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
    return None

solution_dfs = depth_first_graph_search(romania_problem)
if solution_dfs:
    print("Chemin (DFS) =", solution_dfs.solution())
    print("Coût =", solution_dfs.path_cost)
else:
    print("Aucune solution trouvée par DFS.")

Chemin (DFS) = ['Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Craiova', 'Pitesti', 'Bucharest']
Coût = 733


## Recherche informée – Best First Search et A* Search

Pour implémenter une file de priorité, nous utilisons la fonction f(n) pour comparer les noeuds.  
- Greedy Best First Search : f(n) = h(n)  
- A* Search : f(n) = g(n) + h(n) (où g(n) = node.path_cost)

In [25]:
import heapq

class PriorityQueue:
    """
    File de priorité (min-heap). 
    f est une fonction qui renvoie la priorité de l'élément.
    """
    def __init__(self, order='min', f=lambda x: x):
        self.heap = []
        self.order = order
        self.f = f

    def append(self, item):
        # on stocke (f(item), item) dans le heap
        # Python comparera les f(item) ou, en cas d'égalité, comparera item
        # Pour éviter l'erreur 'TypeError: < not supported between instances of Node and Node',
        # on peut ajouter un second critère basé sur un id unique :
        heapq.heappush(self.heap, (self.f(item), id(item), item))

    def pop(self):
        if self.heap:
            return heapq.heappop(self.heap)[2]  # on retourne uniquement l'item
        else:
            raise Exception('La file de priorité est vide.')

    def __contains__(self, item):
        return any(item == elt[2] for elt in self.heap)

    def __len__(self):
        return len(self.heap)

def best_first_graph_search(problem, f):
    """
    Recherche qui sélectionne en priorité le noeud avec la plus petite valeur f(n).
    Couvre Greedy Best First Search (f(n) = h(n)) et A* (f(n) = g(n) + h(n)).
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            else:
                # Si déjà dans frontier avec un coût plus élevé, on pourrait le mettre à jour
                # (AIMA gère un update plus détaillé).
                pass
    return None

def greedy_best_first_search(problem, h=None):
    """Greedy Best First Search : f(n) = h(n)."""
    h = h or problem.h
    return best_first_graph_search(problem, f=lambda n: h(n))

def astar_search(problem, h=None):
    """A* Search : f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_graph_search(problem, f=lambda n: n.path_cost + h(n))

# Exemple d'heuristique simpliste pour la carte : h = 0 pour tout le monde
# (donc A* se réduit à un Uniform Cost Search).
solution_greedy = greedy_best_first_search(romania_problem)
solution_astar  = astar_search(romania_problem)

print("\nGreedy Best First Search :")
if solution_greedy:
    print("Chemin =", solution_greedy.solution())
    print("Coût =", solution_greedy.path_cost)
else:
    print("Aucune solution (Greedy).")

print("\nA* Search :")
if solution_astar:
    print("Chemin =", solution_astar.solution())
    print("Coût =", solution_astar.path_cost)
else:
    print("Aucune solution (A*).")


Greedy Best First Search :
Chemin = ['Sibiu', 'Fagaras', 'Bucharest']
Coût = 450

A* Search :
Chemin = ['Sibiu', 'Fagaras', 'Bucharest']
Coût = 450


## Hill Climbing

Le **Hill Climbing** est un algorithme de recherche locale (ou d’optimisation) qui s’appuie sur la fonction de valeur (ou une heuristique) pour se déplacer vers un meilleur état. Il peut rester bloqué dans un maximum local ou sur un plateau.

L’algorithme général est :
1. On évalue l’état initial.
2. Tant qu’il existe un voisin offrant une meilleure valeur, on se déplace vers ce voisin.
3. On s’arrête lorsqu’aucun voisin n’est meilleur.

Ci-dessous, un exemple de fonction `hill_climbing` (inspiré de la version AIMA) qui prend un `problem` dont la méthode `value(state)` doit renvoyer la valeur (à maximiser).


In [26]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2 in AIMA]"""
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        # On choisit le voisin ayant la valeur la plus élevée.
        neighbor = max(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            # Aucun voisin n'est meilleur : on s'arrête.
            break
        current = neighbor
    return current.state


class HillClimbing1DProblem(Problem):
    def __init__(self, initial, minimum=0, maximum=10):
        super().__init__(initial)
        self.min = minimum
        self.max = maximum

    def actions(self, state):
        # On peut se déplacer de +1 ou -1 si c'est dans les bornes
        acts = []
        if state > self.min:
            acts.append(-1)
        if state < self.max:
            acts.append(+1)
        return acts

    def result(self, state, action):
        # L'action est un déplacement de +1 ou -1
        return state + action

    def goal_test(self, state):
        # Pas de but particulier, on veut juste tester Hill Climbing
        return False

    def value(self, state):
        # Valeur = -(x-7)^2 + 100
        return -(state - 7)**2 + 100


# Testons Hill Climbing sur ce problème 1D
hc_problem_1d = HillClimbing1DProblem(initial=0, minimum=0, maximum=15)
best_state_hc = hill_climbing(hc_problem_1d)
print("Hill Climbing a trouvé l'état =", best_state_hc)
print("Valeur f(x) =", hc_problem_1d.value(best_state_hc))



Hill Climbing a trouvé l'état = 7
Valeur f(x) = 100


## Simulated Annealing

Pour surmonter le blocage dans les optima locaux, **Simulated Annealing** (recuit simulé) introduit une probabilité d’accepter des mouvements « non meilleurs » afin d’explorer l’espace d’états. À haute température, on accepte plus facilement des déplacements défavorables, ce qui favorise l’exploration. Au fur et à mesure que la température décroît, l’algorithme se comporte de plus en plus comme un Hill Climbing classique.

La structure générale :
1. On part d’un état initial.
2. À chaque itération `t`, on calcule une « température » `T = schedule(t)`.
3. On choisit un voisin aléatoire.
4. Si ce voisin est meilleur, on l’accepte. Sinon on l’accepte avec une probabilité `p = exp(delta / T)` où `delta` est la différence de valeur.

Exemple d’implémentation :


In [27]:
import math
import random
import sys

def simulated_annealing(problem, schedule=None):
    """[Figure 4.5 AIMA]
    - problem.value(state) doit renvoyer un score (plus haut = mieux).
    - schedule(t) renvoie la température à l'instant t.
    """
    if schedule is None:
        # Exemple de schedule exponentiel
        schedule = lambda t: (20 * math.exp(-0.005*t) if t < 100 else 0)

    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0:
            current = next_choice
        else:
            # On accepte parfois un voisin pire, selon la proba e^(delta_e / T).
            if random.random() < math.exp(delta_e / T):
                current = next_choice

sa_problem_1d = HillClimbing1DProblem(initial=0, minimum=0, maximum=15)

def custom_schedule(t):
    # Par exemple, décroît exponentiellement
    T0 = 10
    alpha = 0.03
    return T0 * math.exp(-alpha * t)

best_state_sa = simulated_annealing(sa_problem_1d, schedule=custom_schedule)
print("Simulated Annealing a trouvé l'état =", best_state_sa)
print("Valeur f(x) =", sa_problem_1d.value(best_state_sa))



Simulated Annealing a trouvé l'état = 7
Valeur f(x) = 100


## Genetic Algorithm

Les algorithmes génétiques (GA) s’inspirent de la sélection naturelle (Darwin) et de la génétique :
- On maintient une *population* de solutions candidates (individus).
- Chaque individu est codé sous forme de gènes (suite de valeurs).
- À chaque génération :
  - On évalue la *fitness* (qualité) de chaque individu.
  - On effectue la sélection (reproduction) de parents.
  - On *crossover* (mélange) ces parents pour produire des enfants.
  - On opère une mutation aléatoire (faible probabilité).
- On répète jusqu’à atteindre un nombre de générations ou un critère de fitness.

Exemple simplifié ci-dessous :


In [28]:
def init_population(pop_number, gene_pool, state_length):
    """Initialise une population aléatoire de taille pop_number.
    Chaque individu est un tableau de taille state_length,
    où les gènes sont pris dans gene_pool."""
    population = []
    for _ in range(pop_number):
        individual = [random.choice(gene_pool) for __ in range(state_length)]
        population.append(individual)
    return population

def fitness_threshold(fitness_fn, f_thres, population):
    """Renvoie l’individu dont la fitness >= f_thres s’il existe, sinon None."""
    fittest = max(population, key=fitness_fn)
    if fitness_fn(fittest) >= f_thres:
        return fittest
    return None

def recombine(x, y):
    """Point crossover : on coupe à un point aléatoire et on fusionne."""
    c = random.randrange(0, len(x))
    return x[:c] + y[c:]

def mutate(x, gene_pool, pmut=0.1):
    """Mutations : on modifie un gène de x avec une certaine probabilité."""
    if random.random() < pmut:
        idx = random.randrange(len(x))
        x[idx] = random.choice(gene_pool)
    return x

def select(k, population, fitness_fn):
    """Sélection de k individus, en privilégiant les plus fit (roulette wheel)."""
    # Approche simplifiée : on calcule la somme des fitness et on sélectionne par proba.
    total_fitness = sum(fitness_fn(ind) for ind in population)
    chosen = []
    for _ in range(k):
        r = random.uniform(0, total_fitness)
        cumul = 0
        for ind in population:
            cumul += fitness_fn(ind)
            if cumul >= r:
                chosen.append(ind)
                break
    return chosen

def genetic_algorithm(population, fitness_fn, gene_pool, f_thres=None, ngen=1000, pmut=0.1):
    """[Figure 4.8 AIMA] Algorithme génétique basique."""
    for gen in range(ngen):
        population = [
            mutate(recombine(*select(2, population, fitness_fn)), gene_pool, pmut)
            for _ in range(len(population))
        ]
        if f_thres is not None:
            fittest = fitness_threshold(fitness_fn, f_thres, population)
            if fittest:
                return fittest
    # Sinon, on renvoie le meilleur qu’on ait trouvé.
    return max(population, key=fitness_fn)



import random

def fitness_max_one(individual):
    # Somme des bits = nombre de 1
    return sum(individual)

gene_pool = [0, 1]  # On travaille sur des bits
population_size = 20
state_length = 20  # 20 bits par individu

population = init_population(population_size, gene_pool, state_length)
solution_ga = genetic_algorithm(
    population=population,
    fitness_fn=fitness_max_one,
    gene_pool=gene_pool,
    f_thres=state_length,  # On veut 20/20 bits à 1
    ngen=1000,             # nb de générations max
    pmut=0.05              # taux de mutation 5%
)

print("Solution GA =", solution_ga)
print("Fitness =", fitness_max_one(solution_ga))



Solution GA = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Fitness = 20


## AND-OR Graph Search

Dans un environnement non déterministe, l’agent doit trouver un plan conditionnel qui gère toutes les éventualités. L’algorithme d’AND-OR Graph Search (Figure 4.11 dans AIMA) cherche un arbre de plan conditionnel, avec :
- Des nœuds OR : l’agent choisit une action.
- Des nœuds AND : la nature (environnement) propose plusieurs états possibles à la suite d’une action.

L’objectif est de construire un plan (forme arborescente) qui mène à l’état but pour chaque chemin possible.


## Online Search et LRTA*

Les algorithmes de recherche online sont nécessaires si l’agent ne connaît pas la carte ou les transitions à l’avance, ou si l’environnement est dynamique. L’agent intercale des phases de calcul et d’exécution dans le monde. **Online DFS** explore l’espace de manière incrémentale, et **LRTA*** (Learning Real-Time A*) met à jour l’estimation heuristique au fur et à mesure, pour échapper aux minima locaux en environnement inconnu.


# Conclusion

Dans ce notebook, nous avons exploré différents **algorithmes de recherche** basés sur les chapitres 3 et 4 d’_Artificial Intelligence: A Modern Approach_ (AIMA) :

1. **Recherche non informée** :  
   - BFS (Breadth-First Search)  
   - DFS (Depth-First Search)  
   - Variantes telles que Uniform Cost Search, Depth-Limited Search, Iterative Deepening Search.  

2. **Recherche informée** :  
   - Greedy Best First Search  
   - A* Search (et son heuristique)  

3. **Recherche locale et stochastique** :  
   - Hill Climbing, qui peut se bloquer dans des maxima locaux.  
   - Simulated Annealing, qui évite certains blocages via une acceptation probabiliste de mouvements moins bons.  
   - Genetic Algorithm, inspiré de la sélection naturelle, permettant d’évoluer vers de meilleures solutions en mélangeant et en mutant des individus.  

Nous avons illustré leur fonctionnement sur divers problèmes et montré comment implémenter et tester chacun d’eux (avec la classe `Problem` ou des variantes, et des heuristiques adaptées).  

**Bilan :**  
- Les algorithmes de **recherche non informée** fonctionnent pour tous types de problèmes, mais sans guide heuristique, ils peuvent être très coûteux en temps ou mémoire.  
- Les algorithmes de **recherche informée** tirent parti d’une fonction heuristique (A*) ou d’informations partielles (Greedy) pour accélérer la recherche.  
- Les méthodes de **recherche locale et stochastique** (Hill Climbing, Simulated Annealing, GA) sont utiles pour les grands espaces d’états ou les problèmes d’optimisation où l’on ne cherche pas nécessairement un chemin, mais un état optimal ou quasi-optimal.  

Selon le problème, la taille de l’espace d’états et les ressources disponibles, on choisira l’algorithme le plus adapté : la **connaissance du domaine** (heuristique, structure de l’espace d’états) joue un rôle majeur dans l’efficacité de la recherche.  

---

**Références** :  
- S. Russell & P. Norvig. *Artificial Intelligence: A Modern Approach*.  
- Code inspiré d’exemples AIMA en Python.